# Deliverable 3: Strategic Interpretation Dashboard

**Objective:** Build an interactive Plotly Dash dashboard that combines all analytics capabilities — KPI monitoring, trend visualization, scenario projections, anomaly detection, correlation analysis, and AI-generated insights — into a single strategic tool.

**What this notebook demonstrates:**
1. Dashboard architecture and design decisions
2. Preview of all dashboard chart components (rendered inline)
3. How the dashboard integrates data from Deliverables 1 and 2
4. Instructions for launching the live interactive dashboard

---

## Setup

In [ ]:
import sys
sys.path.insert(0, '../..')  # project root

import importlib.util
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

from shared.data_generation.generate import generate_operational_metrics, save_data
from shared.utils.processing import run_full_processing
from shared.utils.plotting import apply_portfolio_style, save_figure, COLORS, SCENARIO_COLORS

def import_from(module_path, module_name):
    spec = importlib.util.spec_from_file_location(module_name, module_path)
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    return mod

pd.set_option('display.float_format', '{:.2f}'.format)
print('Setup complete.')

## 1. Load Data & Run Pipeline

In [ ]:
# Load or generate data
data_path = Path('../data/operational_metrics.csv')
if data_path.exists():
    raw_df = pd.read_csv(data_path, parse_dates=['date'])
else:
    raw_df = generate_operational_metrics()
    save_data(raw_df, '../data')

processed = run_full_processing(raw_df)

modeler = import_from('../../01_scenario_modeling/src/modeler.py', 'modeler')
scenarios = modeler.run_all_scenarios(raw_df)
scenario_summary = modeler.scenarios_endpoint_summary(scenarios)

print(f'Data: {len(raw_df)} rows, {raw_df["facility"].nunique()} facilities, {raw_df["metric"].nunique()} metrics')
print(f'Scenarios: {len(scenario_summary)} projections')

## 2. Dashboard Components Preview

Below we render each dashboard component as a static chart. The live dashboard (`python -m 03_strategic_dashboard.src.app`) adds interactivity, filtering, and AI insight generation.

### 2a. KPI Cards

In [ ]:
# Simulate KPI card data
kpi_metrics = ['production_output', 'quality_rate', 'on_time_delivery_pct', 'defect_rate_ppm']
kpi_data = []
for m in kpi_metrics:
    mdf = raw_df[raw_df['metric'] == m].sort_values('date')
    current = mdf.groupby('facility')['value'].last().mean()
    prev = mdf.groupby('facility')['value'].nth(-2).mean()
    change = ((current - prev) / prev * 100) if prev else 0
    unit = mdf['unit'].iloc[0]
    kpi_data.append({'metric': m.replace('_', ' ').title(), 'current': f'{current:.1f} {unit}', 'mom_change': f'{change:+.1f}%'})

pd.DataFrame(kpi_data)

### 2b. Trend Lines

In [ ]:
display_metrics = ['production_output', 'quality_rate', 'defect_rate_ppm']
df = raw_df[raw_df['metric'].isin(display_metrics)]
cutoff = df['date'].max() - pd.DateOffset(months=12)
df = df[df['date'] >= cutoff]

fig = px.line(df, x='date', y='value', color='facility', facet_row='metric',
              color_discrete_map=COLORS, height=600)
fig = apply_portfolio_style(fig, 'Dashboard: Metric Trends (12 Months)')
fig.update_yaxes(matches=None)
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1].replace('_', ' ').title()))
save_figure(fig, '../outputs/dashboard_trends')
fig.show()

### 2c. Scenario Projections

In [ ]:
# Scenario bar chart
display_scenarios = scenario_summary[scenario_summary['metric'].isin(display_metrics)]

fig = px.bar(
    display_scenarios, x='metric', y='projected_median', color='scenario',
    facet_col='facility', barmode='group',
    color_discrete_map=SCENARIO_COLORS, height=400,
)
fig = apply_portfolio_style(fig, 'Dashboard: Scenario Projections (12 Month Endpoint)')
fig.update_xaxes(tickangle=45)
save_figure(fig, '../outputs/dashboard_scenarios')
fig.show()

In [ ]:
# Scenario fan chart for production output at Plant Alpha
fac, met = 'Plant Alpha', 'production_output'
fig = go.Figure()
for sname, proj in scenarios[fac][met].items():
    color = SCENARIO_COLORS.get(sname, '#888')
    fig.add_trace(go.Scatter(
        x=pd.concat([proj['month'], proj['month'][::-1]]),
        y=pd.concat([proj['p90'], proj['p10'][::-1]]),
        fill='toself', fillcolor=color, opacity=0.15,
        line=dict(width=0), showlegend=False,
    ))
    fig.add_trace(go.Scatter(
        x=proj['month'], y=proj['median'], mode='lines',
        name=sname.replace('_', ' ').title(), line=dict(color=color, width=2),
    ))
fig = apply_portfolio_style(fig, f'Dashboard: {met.replace("_", " ").title()} — {fac} Fan Chart')
fig.update_xaxes(title_text='Months Ahead')
fig.update_layout(height=400)
save_figure(fig, '../outputs/dashboard_fan_chart')
fig.show()

### 2d. Anomaly Detection

In [ ]:
anom = processed['anomalies']
if len(anom) > 0:
    fig = px.scatter(
        anom, x='date', y='z_score', color='facility', symbol='metric',
        size=anom['z_score'].abs(), color_discrete_map=COLORS, height=400,
    )
    fig.add_hline(y=2, line_dash='dash', line_color='orange')
    fig.add_hline(y=-2, line_dash='dash', line_color='orange')
    fig = apply_portfolio_style(fig, 'Dashboard: Anomaly Detection')
    save_figure(fig, '../outputs/dashboard_anomalies')
    fig.show()

### 2e. Correlation Heatmap & Facility Radar

In [ ]:
# Correlation heatmap for Plant Alpha
fac_corr = processed['correlations'][processed['correlations']['facility'] == 'Plant Alpha']
metrics = sorted(set(fac_corr['metric_1']) | set(fac_corr['metric_2']))
matrix = pd.DataFrame(1.0, index=metrics, columns=metrics)
for _, row in fac_corr.iterrows():
    matrix.loc[row['metric_1'], row['metric_2']] = row['correlation']
    matrix.loc[row['metric_2'], row['metric_1']] = row['correlation']

fig = px.imshow(matrix, text_auto='.2f', color_continuous_scale='RdBu_r', zmin=-1, zmax=1)
fig = apply_portfolio_style(fig, 'Dashboard: Correlation Heatmap — Plant Alpha')
fig.update_layout(height=500)
save_figure(fig, '../outputs/dashboard_correlations')
fig.show()

In [ ]:
# Facility radar
summary = processed['summary']
radar = summary.pivot_table(index='metric', columns='facility', values='mean')
norm = radar.apply(lambda x: (x - x.min()) / (x.max() - x.min()) * 100 if x.max() != x.min() else 50, axis=1)

fig = go.Figure()
for fac in norm.columns:
    vals = norm[fac].tolist() + [norm[fac].iloc[0]]
    cats = norm.index.tolist() + [norm.index[0]]
    fig.add_trace(go.Scatterpolar(
        r=vals, theta=cats, fill='toself', name=fac, opacity=0.6,
        line=dict(color=COLORS.get(fac)),
    ))
fig = apply_portfolio_style(fig, 'Dashboard: Facility Comparison Radar')
fig.update_layout(height=500, polar=dict(radialaxis=dict(visible=True, range=[0, 100])))
save_figure(fig, '../outputs/dashboard_radar')
fig.show()

## 3. Launching the Live Dashboard

To run the full interactive dashboard with real-time filtering and AI insights:

```bash
# From the project root directory:
python main.py dashboard

# Or directly:
python 03_strategic_dashboard/src/app.py

# Then open: http://localhost:8050
```

The live dashboard adds:
- **Real-time filtering** by facility, metric, and time range
- **AI Insights panel** — click "Generate Insights" to call Claude API
- **Tabbed interface** for clean navigation between analysis views
- **Responsive KPI cards** with month-over-month change indicators

---

## Summary

| Dashboard Component | Data Source | Interactivity |
|---|---|---|
| KPI Cards | Raw metrics (latest vs. previous month) | Filter by facility, time range |
| Trend Charts | Raw time series | Filter by metric, facility, time range |
| Scenario Projections | Monte Carlo simulation (Deliverable 1) | Filter by facility, metric |
| Fan Charts | Simulation percentile bands | Select facility and metric |
| Anomaly Plot | Z-score detection | Filter by facility |
| Correlation Heatmap | Pearson correlations | Select facility |
| Facility Radar | Normalized summary stats | Filter by facility |
| AI Insights Panel | Claude API (Deliverable 2) | On-demand generation |

**Tech stack:** Plotly Dash + Bootstrap (Flatly theme), Plotly.js charts, Claude API integration

**Next:** See [04_documentation](../../04_documentation/) for guides and training materials.